В этом коде я создаю текстовый csv файл следующейго содержания:

данные из html файлов отдела FTL в виде текстовой базы данных
Вид:

целевые столбцы Query, Answer, Department, Link, где

Query = названию файла html (по возможности без текстового мусора)

Answer содержит всю содержательную часть файлов папки FTL, соответствующую Query в этой строке. Табличные данные вида вопрос-ответ были распарсены немного по-другому, строки таких таблиц стали отдельными строками в нашем csv, в этом случае создано несколько строк с одинаковым Query, соответствуюшим названию файла (Напрмер, Частые Вопросы), а внутри Answer будет лишь один смысловой вопрос-ответ, как отдельная смысловая еденица.

Department - отдел (=FTL)

Link - ссылка на конфлюенс

Загрузка необходимых библиотек. BeautifulSoup помогает распарсить данные из html файлов, os помогает работать с файлами, папками, переименованием, pandas - библиотека для обработки и анализа данных, fuzzywuzzy используется для нахождения наиболее похожих строк, может использоваться для нахождения опечаток

In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
!pip install fuzzywuzzy[speedup]
from fuzzywuzzy import process

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
directory = '/content/drive/MyDrive/PEK/FTL'

In [4]:
paragraphs_dataframes = []

In [5]:
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)

    # Check if the file exists and is an HTML file
    if os.path.isfile(file_path) and filename.endswith('.html'):
        # Load the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract all paragraphs and concatenate their text
        paragraphs = soup.find_all('p')
        tables = soup.find_all('td')
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
        table_items_text = ' '.join(table.get_text(strip=True) for table in tables)
        combined_text = paragraphs_text + ' ' + table_items_text

        # Create a DataFrame for the concatenated paragraphs
        df_paragraphs = pd.DataFrame([[os.path.splitext(filename)[0], combined_text]],
                                     columns=['Query', 'Answer'])

        # Append the DataFrame to the list
        paragraphs_dataframes.append(df_paragraphs)

In [6]:
combined_paragraphs_df = []
combined_paragraphs_df = pd.concat(paragraphs_dataframes, ignore_index=True)

In [7]:
display(combined_paragraphs_df)

,Query,Answer
0,АРМ логиста FTL - в проде - Full Truck Load - ...,FTL-4-Получение подробных данных проблемы…СТАТ...
1,АРМ менеджера FTL - в проде - Full Truck Load ...,FTL-32-Получение подробных данных проблемы…СТА...
2,Алгоритм действий сотрудников по работе с за...,FTL-62-Получение подробных данных проблемы…СТА...
3,Алгоритм действий сотрудников по работе с за...,Отслеживать изменения статусов электронной зая...
4,Документ Рейс - MVP - Full Truck Load - База ...,Основной эпик -EPIC-9968-Получение подробных д...
5,Загрузка адресного классификатора в ИС 1С_ TMS...,Целевая группа – администратор ИС 1С: TMS Логи...
6,Загрузка прайса по услугам FTL в 1С_ TMS Логи...,FTL-33-Получение подробных данных проблемы…СТА...
7,ИС 1С_ TMS настройка приложения - в проде - F...,Целевая группа – пользователи и администратор ...
8,Калькулятор FTL - MVP (но еще не обновлена) - ...,https://evgenia-petrova.atlassian.net/browse/F...
9,Карточка контрагента. Создание адреса доставки...,FTL-128-Получение подробных данных проблемы…СТ...


In [9]:
patterns_to_remove = ['…СТАТУС', 'ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.']
for pattern in patterns_to_remove:
    combined_paragraphs_df['Answer'] = combined_paragraphs_df['Answer'].str.replace(pattern, '', regex=False)

display(combined_paragraphs_df)

,Query,Answer
0,АРМ логиста FTL - в проде - Full Truck Load - ...,FTL-4-Получение подробных данных проблемы Целе...
1,АРМ менеджера FTL - в проде - Full Truck Load ...,FTL-32-Получение подробных данных проблемы Со...
2,Алгоритм действий сотрудников по работе с за...,FTL-62-Получение подробных данных проблемы АРМ...
3,Алгоритм действий сотрудников по работе с за...,Отслеживать изменения статусов электронной зая...
4,Документ Рейс - MVP - Full Truck Load - База ...,Основной эпик -EPIC-9968-Получение подробных д...
5,Загрузка адресного классификатора в ИС 1С_ TMS...,Целевая группа – администратор ИС 1С: TMS Логи...
6,Загрузка прайса по услугам FTL в 1С_ TMS Логи...,FTL-33-Получение подробных данных проблемы Цел...
7,ИС 1С_ TMS настройка приложения - в проде - F...,Целевая группа – пользователи и администратор ...
8,Калькулятор FTL - MVP (но еще не обновлена) - ...,https://evgenia-petrova.atlassian.net/browse/F...
9,Карточка контрагента. Создание адреса доставки...,FTL-128-Получение подробных данных проблемы Це...


In [10]:
# Texts to remove
unwanted_endings = [
    ' - Full Truck Load - База знаний'
]

# Function to remove unwanted text endings from 'File' column
def remove_unwanted_endings(file_name):
    for ending in unwanted_endings:
        if file_name.endswith(ending):
            return file_name[:-len(ending)].strip()
    return file_name

In [11]:
combined_paragraphs_df['Query'] = combined_paragraphs_df['Query'].apply(remove_unwanted_endings)

In [12]:
display(combined_paragraphs_df)

,Query,Answer
0,АРМ логиста FTL - в проде,FTL-4-Получение подробных данных проблемы Целе...
1,АРМ менеджера FTL - в проде,FTL-32-Получение подробных данных проблемы Со...
2,Алгоритм действий сотрудников по работе с за...,FTL-62-Получение подробных данных проблемы АРМ...
3,Алгоритм действий сотрудников по работе с за...,Отслеживать изменения статусов электронной зая...
4,Документ Рейс - MVP,Основной эпик -EPIC-9968-Получение подробных д...
5,Загрузка адресного классификатора в ИС 1С_ TMS...,Целевая группа – администратор ИС 1С: TMS Логи...
6,Загрузка прайса по услугам FTL в 1С_ TMS Логи...,FTL-33-Получение подробных данных проблемы Цел...
7,ИС 1С_ TMS настройка приложения - в проде,Целевая группа – пользователи и администратор ...
8,Калькулятор FTL - MVP (но еще не обновлена),https://evgenia-petrova.atlassian.net/browse/F...
9,Карточка контрагента. Создание адреса доставки,FTL-128-Получение подробных данных проблемы Це...


In [13]:
 # Texts to remove
unwanted_endings = [
    ' - в проде'
]

# Function to remove unwanted text endings from 'File' column
def remove_unwanted_endings(file_name):
    for ending in unwanted_endings:
        if file_name.endswith(ending):
            return file_name[:-len(ending)].strip()
    return file_name

In [14]:
combined_paragraphs_df['Query'] = combined_paragraphs_df['Query'].apply(remove_unwanted_endings)

In [15]:
display(combined_paragraphs_df)

,Query,Answer
0,АРМ логиста FTL,FTL-4-Получение подробных данных проблемы Целе...
1,АРМ менеджера FTL,FTL-32-Получение подробных данных проблемы Со...
2,Алгоритм действий сотрудников по работе с за...,FTL-62-Получение подробных данных проблемы АРМ...
3,Алгоритм действий сотрудников по работе с за...,Отслеживать изменения статусов электронной зая...
4,Документ Рейс - MVP,Основной эпик -EPIC-9968-Получение подробных д...
5,Загрузка адресного классификатора в ИС 1С_ TMS...,Целевая группа – администратор ИС 1С: TMS Логи...
6,Загрузка прайса по услугам FTL в 1С_ TMS Логи...,FTL-33-Получение подробных данных проблемы Цел...
7,ИС 1С_ TMS настройка приложения,Целевая группа – пользователи и администратор ...
8,Калькулятор FTL - MVP (но еще не обновлена),https://evgenia-petrova.atlassian.net/browse/F...
9,Карточка контрагента. Создание адреса доставки,FTL-128-Получение подробных данных проблемы Це...


In [16]:
links=pd.read_csv('/content/drive/MyDrive/PEK/FTL.txt', sep='/t', header=None)
display(links) #links

<ipython-input-16-a5202bbc43fc>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links=pd.read_csv('/content/drive/MyDrive/PEK/FTL.txt', sep='/t', header=None)


,0
0,АРМ логиста FTL - в проде;http://confluence/pa...
1,АРМ менеджера FTL - в проде;http://confluence/...
2,Алгоритм действий сотрудников по работе с заяв...
3,Алгоритм действий сотрудников по работе с заяв...
4,Документ Рейс - MVP;http://confluence/pages/vi...
5,Загрузка адресного классификатора в ИС 1С: TMS...
6,Загрузка прайса по услугам FTL в 1С: TMS Логис...
7,ИС 1С: TMS настройка приложения - в проде;http...
8,Калькулятор FTL - MVP (но еще не обновлена);ht...
9,Карточка контрагента. Создание адреса доставки...


In [17]:
# Split the unnamed column by the last ";"

# Function to split by the last semicolon
def split_by_last_semicolon(text):
    parts = text.rsplit(';', 1)
    return parts if len(parts) == 2 else [text, '']

# Apply the function to split the unnamed column
links_split = links.iloc[:, 0].apply(split_by_last_semicolon).apply(pd.Series)

# Rename the columns
links_split.columns = ['File', 'Link']

display(links_split)

,File,Link
0,АРМ логиста FTL - в проде,http://confluence/pages/viewpage.action?pageId...
1,АРМ менеджера FTL - в проде,http://confluence/pages/viewpage.action?pageId...
2,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...
3,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...
4,Документ Рейс - MVP,http://confluence/pages/viewpage.action?pageId...
5,Загрузка адресного классификатора в ИС 1С: TMS...,http://confluence/pages/viewpage.action?pageId...
6,Загрузка прайса по услугам FTL в 1С: TMS Логис...,http://confluence/pages/viewpage.action?pageId...
7,ИС 1С: TMS настройка приложения - в проде,http://confluence/pages/viewpage.action?pageId...
8,Калькулятор FTL - MVP (но еще не обновлена),http://confluence/pages/viewpage.action?pageId...
9,Карточка контрагента. Создание адреса доставки,http://confluence/pages/viewpage.action?pageId...


In [18]:
def get_best_match(query, choices, threshold=80):
    result = process.extractOne(query, choices)
    if result:
        best_match = result[0]
        score = result[1]
        if score >= threshold:
            return best_match
    return None

In [21]:
links_split['Best_Match'] = links_split['File'].apply(lambda x: get_best_match(x, combined_paragraphs_df['Query']))

In [22]:
display(links_split)

,File,Link,Best_Match
0,АРМ логиста FTL - в проде,http://confluence/pages/viewpage.action?pageId...,АРМ логиста FTL
1,АРМ менеджера FTL - в проде,http://confluence/pages/viewpage.action?pageId...,АРМ менеджера FTL
2,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...,Алгоритм действий сотрудников по работе с за...
3,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...,Алгоритм действий сотрудников по работе с за...
4,Документ Рейс - MVP,http://confluence/pages/viewpage.action?pageId...,Документ Рейс - MVP
5,Загрузка адресного классификатора в ИС 1С: TMS...,http://confluence/pages/viewpage.action?pageId...,Загрузка адресного классификатора в ИС 1С_ TMS...
6,Загрузка прайса по услугам FTL в 1С: TMS Логис...,http://confluence/pages/viewpage.action?pageId...,АРМ логиста FTL
7,ИС 1С: TMS настройка приложения - в проде,http://confluence/pages/viewpage.action?pageId...,Алгоритм действий сотрудников по работе с за...
8,Калькулятор FTL - MVP (но еще не обновлена),http://confluence/pages/viewpage.action?pageId...,Калькулятор FTL - MVP (но еще не обновлена)
9,Карточка контрагента. Создание адреса доставки,http://confluence/pages/viewpage.action?pageId...,Карточка контрагента. Создание адреса доставки


In [23]:
links_split.at[6, 'Best_Match'] = 'Загрузка прайса по услугам FTL в 1С_ TMS Логистика'
links_split.at[7, 'Best_Match'] = 'ИС 1С_ TMS настройка приложения'

In [24]:
display(links_split)

,File,Link,Best_Match
0,АРМ логиста FTL - в проде,http://confluence/pages/viewpage.action?pageId...,АРМ логиста FTL
1,АРМ менеджера FTL - в проде,http://confluence/pages/viewpage.action?pageId...,АРМ менеджера FTL
2,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...,Алгоритм действий сотрудников по работе с за...
3,Алгоритм действий сотрудников по работе с заяв...,http://confluence/pages/viewpage.action?pageId...,Алгоритм действий сотрудников по работе с за...
4,Документ Рейс - MVP,http://confluence/pages/viewpage.action?pageId...,Документ Рейс - MVP
5,Загрузка адресного классификатора в ИС 1С: TMS...,http://confluence/pages/viewpage.action?pageId...,Загрузка адресного классификатора в ИС 1С_ TMS...
6,Загрузка прайса по услугам FTL в 1С: TMS Логис...,http://confluence/pages/viewpage.action?pageId...,Загрузка прайса по услугам FTL в 1С_ TMS Логи...
7,ИС 1С: TMS настройка приложения - в проде,http://confluence/pages/viewpage.action?pageId...,ИС 1С_ TMS настройка приложения
8,Калькулятор FTL - MVP (но еще не обновлена),http://confluence/pages/viewpage.action?pageId...,Калькулятор FTL - MVP (но еще не обновлена)
9,Карточка контрагента. Создание адреса доставки,http://confluence/pages/viewpage.action?pageId...,Карточка контрагента. Создание адреса доставки


In [27]:
merged_ftl_df=[]
# Create a function to merge based on the custom condition
def custom_merge(df1, df2):
    merged_data = []
    for _, row1 in df1.iterrows():
        query = row1['Query']
        matching_link = None
        for _, row2 in df2.iterrows():
            if query.startswith(row2['Best_Match']):
                matching_link = row2['Link']
                break
        merged_data.append({
            'Query': row1['Query'],
            'Answer': row1['Answer'],
            'Link': matching_link
        })
    return pd.DataFrame(merged_data)

# Merge DataFrames using the custom function
merged_ftl_df = custom_merge(combined_paragraphs_df, links_split)

In [28]:
merged_ftl_df['Department'] = 'FTL'

In [29]:
merged_ftl_df.to_csv('/content/drive/MyDrive/merged_ftl_df.csv', index=False)